# Cyclone Dust Collector Predictor (engineering starter notebook)

This notebook is a **more rigorous starting point** for cyclone screening calculations.

## Predicts
- Pressure drop (`Pa`, `kPa`, `psig`)
- Cut size `d50` (µm)
- Grade efficiency curve and estimated overall collection efficiency (%)

## Why this version is better
- Converts **standard flow (SLPM)** to actual volumetric flow at operating `T, P`.
- Uses **species-wise viscosity** via Sutherland correlation and **Wilke mixing rule**.
- Keeps all geometry as explicit user inputs (with Stairmand defaults prefilled).
- Adds sensitivity sweeps and clear calibration guidance.

> This remains a reduced-order model (not CFD). Treat as a design-screening tool and calibrate constants with test data.


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# ---------- constants ----------
R = 8.314462618  # J/mol/K
PA_PER_PSIG = 6894.757293168
MICRON = 1e-6

# Standard condition used for SLPM convention in this notebook
T_STD_K = 273.15
P_STD_PA = 101325.0


## 1) Inputs (edit these)

In [ ]:
inputs = {
    # operating point
    "flow_slpm": 50.0,             # standard L/min
    "T_C": 200.0,                  # gas temperature (C)
    "P_abs_Pa": 101325.0,          # absolute pressure (Pa)

    # gas composition (mole fraction, auto-normalized)
    "y_Ar": 0.50,
    "y_CH4": 0.25,
    "y_H2": 0.25,

    # solids
    "particle_density_kg_m3": 500.0,
    "bulk_density_g_L": 30.0,      # informational here; not aerosol concentration

    # assumed feed PSD (replace with measured data when possible)
    "d63_um": 12.0,                # Rosin-Rammler scale
    "rr_n": 2.2,                   # Rosin-Rammler shape

    # cyclone size + tunables
    "D_m": 0.05,                   # body diameter
    "Euler_number": 6.0,           # pressure-drop correlation coefficient
    "k_cut": 0.65,                 # cut-size tuning coefficient
}

# Stairmand-like high-efficiency starter ratios (editable)
ratios = {
    "a_over_D": 0.50,  # inlet height
    "b_over_D": 0.20,  # inlet width
    "De_over_D": 0.50, # vortex finder diameter
    "S_over_D": 0.50,  # vortex finder insertion
    "h_over_D": 1.50,  # barrel height
    "H_over_D": 4.00,  # total height
    "B_over_D": 0.375, # dust outlet diameter
}

pd.Series(inputs).to_frame("value")

## 2) Gas-property model (Wilke + Sutherland)

In [ ]:
# Species property constants
MW = {  # kg/mol
    "Ar": 39.948e-3,
    "CH4": 16.043e-3,
    "H2": 2.016e-3,
}

# Sutherland-form constants: mu = mu0*(T/T0)^1.5 * (T0 + S)/(T + S)
# Values are screening-level engineering constants.
SUTH = {
    "Ar":  {"mu0": 2.23e-5, "T0": 300.0, "S": 144.0},
    "CH4": {"mu0": 1.10e-5, "T0": 300.0, "S": 199.0},
    "H2":  {"mu0": 8.76e-6, "T0": 300.0, "S": 72.0},
}

def normalize_comp(y):
    s = sum(y.values())
    if s <= 0:
        raise ValueError("Gas composition sum must be > 0")
    return {k: v/s for k, v in y.items()}

def species_mu(T_K, sp):
    c = SUTH[sp]
    return c["mu0"] * (T_K/c["T0"])**1.5 * (c["T0"] + c["S"]) / (T_K + c["S"])

def gas_mw(y):
    y = normalize_comp(y)
    return sum(y[k]*MW[k] for k in y)

def gas_density_ideal(y, T_K, P_abs_Pa):
    return P_abs_Pa * gas_mw(y) / (R*T_K)

def wilke_mixture_viscosity(y, T_K):
    y = normalize_comp(y)
    species = list(y.keys())
    mu = {k: species_mu(T_K, k) for k in species}

    phi = {i: {} for i in species}
    for i in species:
        for j in species:
            num = (1 + (mu[i]/mu[j])**0.5 * (MW[j]/MW[i])**0.25)**2
            den = (8*(1 + MW[i]/MW[j]))**0.5
            phi[i][j] = num / den

    return sum(y[i]*mu[i] / sum(y[j]*phi[i][j] for j in species) for i in species)

def slpm_to_actual_m3_s(flow_slpm, T_oper_K, P_oper_Pa, T_std_K=T_STD_K, P_std_Pa=P_STD_PA):
    Q_std = flow_slpm * 1e-3 / 60.0
    # from ideal gas: Q_actual = Q_std*(T_oper/T_std)*(P_std/P_oper)
    return Q_std * (T_oper_K/T_std_K) * (P_std_Pa/P_oper_Pa)


## 3) Cyclone equations

In [ ]:
def cyclone_dimensions(D, ratios):
    return {
        "D": D,
        "a": ratios["a_over_D"]*D,
        "b": ratios["b_over_D"]*D,
        "De": ratios["De_over_D"]*D,
        "S": ratios["S_over_D"]*D,
        "h": ratios["h_over_D"]*D,
        "H": ratios["H_over_D"]*D,
        "B": ratios["B_over_D"]*D,
    }

def inlet_area(dims):
    return dims["a"]*dims["b"]

def inlet_velocity(Q_actual_m3_s, dims):
    return Q_actual_m3_s / inlet_area(dims)

def pressure_drop_pa(rho_g, V_in, euler_number):
    # DeltaP = Eu * (rho * V^2 / 2)
    return euler_number * 0.5 * rho_g * V_in**2

def lapple_cut_size_m(dims, rho_p, rho_g, mu_g, Q_actual_m3_s, k_cut=0.65):
    # Lapple-inspired scaling (screening-level)
    V_in = inlet_velocity(Q_actual_m3_s, dims)
    b = dims["b"]
    return np.sqrt((k_cut*mu_g*b)/(max(rho_p-rho_g, 1e-12)*V_in))

def rosin_rammler_cdf(d_um, d63_um, n):
    d = np.maximum(np.asarray(d_um, float), 1e-12)
    return 1.0 - np.exp(-(d/d63_um)**n)

# Use np.trapezoid (NumPy >= 2.0) with fallback to np.trapz for older versions
_trapz = getattr(np, "trapezoid", None) or np.trapz

def overall_efficiency_from_d50(d50_um, d63_um, n, alpha=2.0, n_pts=800):
    d = np.geomspace(0.1, 300, n_pts)
    F = rosin_rammler_cdf(d, d63_um, n)
    dF = np.gradient(F, d)
    dF = np.clip(dF, 0, None)
    area = _trapz(dF, d)
    if area <= 0:
        return np.nan, d, dF*0, dF*0
    pdf = dF/area

    eta_d = 1/(1 + (d50_um/np.maximum(d, 1e-9))**alpha)
    eta_overall = _trapz(eta_d*pdf, d)
    return float(np.clip(eta_overall, 0, 1)), d, pdf, eta_d

## 4) Baseline run

In [ ]:
T_K = inputs["T_C"] + 273.15
y = {"Ar": inputs["y_Ar"], "CH4": inputs["y_CH4"], "H2": inputs["y_H2"]}

Q_actual = slpm_to_actual_m3_s(inputs["flow_slpm"], T_K, inputs["P_abs_Pa"])
rho_g = gas_density_ideal(y, T_K, inputs["P_abs_Pa"])
mu_g = wilke_mixture_viscosity(y, T_K)

dims = cyclone_dimensions(inputs["D_m"], ratios)
V_in = inlet_velocity(Q_actual, dims)

dP_pa = pressure_drop_pa(rho_g, V_in, inputs["Euler_number"])
dP_psig = dP_pa/PA_PER_PSIG

d50_um = lapple_cut_size_m(
    dims=dims,
    rho_p=inputs["particle_density_kg_m3"],
    rho_g=rho_g,
    mu_g=mu_g,
    Q_actual_m3_s=Q_actual,
    k_cut=inputs["k_cut"],
)/MICRON

eta, d_um, pdf, eta_d = overall_efficiency_from_d50(
    d50_um=d50_um,
    d63_um=inputs["d63_um"],
    n=inputs["rr_n"],
)

# --- Geometry table with descriptions and inches ---
M_TO_IN = 1000 / 25.4  # metres -> inches

dim_descriptions = {
    "D":  "Cyclone body ID",
    "a":  "Inlet height",
    "b":  "Inlet width",
    "De": "Vortex finder ID",
    "S":  "Vortex finder insertion depth",
    "h":  "Barrel (cylinder) height",
    "H":  "Total cyclone height",
    "B":  "Dust outlet ID",
}

geom_df = pd.DataFrame({
    "Description": {k: dim_descriptions[k] for k in dims},
    "m":           {k: v for k, v in dims.items()},
    "in":          {k: v * M_TO_IN for k, v in dims.items()},
})
geom_df.index.name = "Dim"

# --- Summary table rounded to 2 significant figures ---
def sig_figs(x, n=2):
    """Round x to n significant figures."""
    if x == 0 or np.isnan(x):
        return x
    return round(x, -int(np.floor(np.log10(abs(x)))) + (n - 1))

summary_raw = {
    "Flow (SLPM, standard)": inputs["flow_slpm"],
    "Flow (m3/s, actual)": Q_actual,
    "Temperature (C)": inputs["T_C"],
    "Pressure abs (kPa)": inputs["P_abs_Pa"]/1000,
    "Gas density (kg/m3)": rho_g,
    "Gas viscosity (Pa*s)": mu_g,
    "Inlet velocity (m/s)": V_in,
    "Pressure drop (Pa)": dP_pa,
    "Pressure drop (psig)": dP_psig,
    "Cut size d50 (um)": d50_um,
    "Collection efficiency (%)": 100*eta,
}
summary = pd.Series({k: sig_figs(v) for k, v in summary_raw.items()}).to_frame("value")

display(geom_df)
display(summary)

In [ ]:
fig, ax = plt.subplots(1,2, figsize=(11,4))

ax[0].plot(d_um, pdf)
ax[0].set_xscale("log")
ax[0].set_title("Assumed feed PSD (Rosin-Rammler)")
ax[0].set_xlabel("Particle diameter (um)")
ax[0].set_ylabel("Normalized PDF")
ax[0].grid(True, which="both", ls="--", alpha=0.5)

ax[1].plot(d_um, 100*eta_d)
ax[1].axvline(d50_um, ls="--", c="r", label=f"d50 = {d50_um:.2f} um")
ax[1].set_xscale("log")
ax[1].set_title("Grade efficiency")
ax[1].set_xlabel("Particle diameter (um)")
ax[1].set_ylabel("Capture efficiency (%)")
ax[1].legend()
ax[1].grid(True, which="both", ls="--", alpha=0.5)

plt.tight_layout()
plt.show()

## 5) Sensitivity sweeps

Sweeps across **cyclone body diameter**, **flow rate**, **temperature**, and **Euler number** to visualize how pressure drop and collection efficiency trade off.

In [ ]:
def evaluate_case(flow_slpm=None, D_m=None, T_C=None, euler_number=None):
    p = dict(inputs)
    if flow_slpm is not None: p["flow_slpm"] = float(flow_slpm)
    if D_m is not None: p["D_m"] = float(D_m)
    if T_C is not None: p["T_C"] = float(T_C)
    if euler_number is not None: p["Euler_number"] = float(euler_number)

    T_K = p["T_C"] + 273.15
    y = {"Ar": p["y_Ar"], "CH4": p["y_CH4"], "H2": p["y_H2"]}

    Q_actual = slpm_to_actual_m3_s(p["flow_slpm"], T_K, p["P_abs_Pa"])
    rho_g = gas_density_ideal(y, T_K, p["P_abs_Pa"])
    mu_g = wilke_mixture_viscosity(y, T_K)

    dims = cyclone_dimensions(p["D_m"], ratios)
    V_in = inlet_velocity(Q_actual, dims)
    dP_pa = pressure_drop_pa(rho_g, V_in, p["Euler_number"])
    dP_psig = dP_pa / PA_PER_PSIG
    d50_um = lapple_cut_size_m(dims, p["particle_density_kg_m3"], rho_g, mu_g, Q_actual, p["k_cut"]) / MICRON
    eta, *_ = overall_efficiency_from_d50(d50_um, p["d63_um"], p["rr_n"])

    return {
        "flow_slpm": p["flow_slpm"],
        "D_m": p["D_m"],
        "D_in": p["D_m"] * M_TO_IN,
        "T_C": p["T_C"],
        "Eu": p["Euler_number"],
        "V_in_m_s": V_in,
        "dP_Pa": dP_pa,
        "dP_psig": dP_psig,
        "d50_um": d50_um,
        "eff_%": 100 * eta,
    }

# ── Sweep 1: Cyclone body diameter ──
diameters = np.linspace(0.020, 0.10, 25)
sweep_D = pd.DataFrame([evaluate_case(D_m=d) for d in diameters])

# ── Sweep 2: Flow rate ──
flows = np.linspace(10, 200, 25)
sweep_Q = pd.DataFrame([evaluate_case(flow_slpm=q) for q in flows])

# ── Sweep 3: Temperature ──
temps = np.linspace(25, 500, 25)
sweep_T = pd.DataFrame([evaluate_case(T_C=t) for t in temps])

# ── Sweep 4: Euler number ──
eulers = np.linspace(2, 20, 25)
sweep_Eu = pd.DataFrame([evaluate_case(euler_number=e) for e in eulers])

# ── Tables ──
print("=== Diameter sweep ===")
display(sweep_D[["D_m", "D_in", "V_in_m_s", "dP_Pa", "dP_psig", "d50_um", "eff_%"]].round(3))

print("\n=== Flow-rate sweep ===")
display(sweep_Q[["flow_slpm", "V_in_m_s", "dP_Pa", "dP_psig", "d50_um", "eff_%"]].round(3))

print("\n=== Temperature sweep ===")
display(sweep_T[["T_C", "V_in_m_s", "dP_Pa", "dP_psig", "d50_um", "eff_%"]].round(3))

print("\n=== Euler-number sweep ===")
display(sweep_Eu[["Eu", "dP_Pa", "dP_psig", "eff_%"]].round(3))

In [ ]:
# ── Sweep plots ──
fig, axes = plt.subplots(2, 4, figsize=(18, 9))

# --- Row 1: Pressure drop ---
# Diameter sweep
axes[0, 0].plot(sweep_D["D_in"], sweep_D["dP_psig"], "b-o", ms=3)
axes[0, 0].set_xlabel("Body diameter D (in)")
axes[0, 0].set_ylabel("Pressure drop (psig)")
axes[0, 0].set_title("dP vs Diameter")
axes[0, 0].grid(True, which="both", ls="--", alpha=0.5)

# Flow sweep
axes[0, 1].plot(sweep_Q["flow_slpm"], sweep_Q["dP_psig"], "b-o", ms=3)
axes[0, 1].set_xlabel("Flow (SLPM)")
axes[0, 1].set_ylabel("Pressure drop (psig)")
axes[0, 1].set_title("dP vs Flow")
axes[0, 1].grid(True, which="both", ls="--", alpha=0.5)

# Temperature sweep
axes[0, 2].plot(sweep_T["T_C"], sweep_T["dP_psig"], "b-o", ms=3)
axes[0, 2].set_xlabel("Temperature (C)")
axes[0, 2].set_ylabel("Pressure drop (psig)")
axes[0, 2].set_title("dP vs Temperature")
axes[0, 2].grid(True, which="both", ls="--", alpha=0.5)

# Euler sweep
axes[0, 3].plot(sweep_Eu["Eu"], sweep_Eu["dP_psig"], "b-o", ms=3)
axes[0, 3].set_xlabel("Euler number")
axes[0, 3].set_ylabel("Pressure drop (psig)")
axes[0, 3].set_title("dP vs Euler number")
axes[0, 3].grid(True, which="both", ls="--", alpha=0.5)

# --- Row 2: Collection efficiency ---
# Diameter sweep
axes[1, 0].plot(sweep_D["D_in"], sweep_D["eff_%"], "g-s", ms=3)
axes[1, 0].set_xlabel("Body diameter D (in)")
axes[1, 0].set_ylabel("Collection efficiency (%)")
axes[1, 0].set_title("Efficiency vs Diameter")
axes[1, 0].grid(True, which="both", ls="--", alpha=0.5)

# Flow sweep
axes[1, 1].plot(sweep_Q["flow_slpm"], sweep_Q["eff_%"], "g-s", ms=3)
axes[1, 1].set_xlabel("Flow (SLPM)")
axes[1, 1].set_ylabel("Collection efficiency (%)")
axes[1, 1].set_title("Efficiency vs Flow")
axes[1, 1].grid(True, which="both", ls="--", alpha=0.5)

# Temperature sweep
axes[1, 2].plot(sweep_T["T_C"], sweep_T["eff_%"], "g-s", ms=3)
axes[1, 2].set_xlabel("Temperature (C)")
axes[1, 2].set_ylabel("Collection efficiency (%)")
axes[1, 2].set_title("Efficiency vs Temperature")
axes[1, 2].grid(True, which="both", ls="--", alpha=0.5)

# Euler sweep (efficiency is constant vs Eu since Eu only affects dP, not d50)
axes[1, 3].plot(sweep_Eu["Eu"], sweep_Eu["eff_%"], "g-s", ms=3)
axes[1, 3].set_xlabel("Euler number")
axes[1, 3].set_ylabel("Collection efficiency (%)")
axes[1, 3].set_title("Efficiency vs Euler number")
axes[1, 3].grid(True, which="both", ls="--", alpha=0.5)

fig.suptitle("Sensitivity Sweeps — Pressure Drop (top) and Collection Efficiency (bottom)",
             fontsize=13, fontweight="bold", y=1.01)
plt.tight_layout()
plt.show()

## What to do next
1. **Run all cells** to generate the baseline outputs.
2. Update geometry and operating conditions to your intended design point.
3. Replace assumed PSD parameters (`d63_um`, `rr_n`) with measured particle-size data.
4. Calibrate `Euler_number` and `k_cut` using one or more known test points.
5. Use the sweep cell to select a geometry balancing efficiency and pressure-drop budget.

If you want, the next upgrade can be a **measured-PSD input cell** (CSV import + interpolation) so overall efficiency is based directly on lab data instead of a Rosin-Rammler fit.
